In [14]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


from methods.methods import(
    method_optimal, 
    method_z,
    method_contrained,
    method_dual, 
    method_incentive
)

from methods.utils import(
    expected_payoff, 
    inititial_sol,
    concat_df
)

from tqdm import tqdm

# Random Test Case

In [2]:
#Collecting all the required dataframes and dropping the first column

path = "../Generated cases/case1.xlsx"
SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
DistanceSiteTeam = DistanceSiteTeam.drop("Unnamed: 0", axis=1)
feasible_locations = (DistanceSiteTeam <= 60).astype(int)
Site = pd.read_excel(path, sheet_name="Site")
Team = pd.read_excel(path, sheet_name="Team")
muST = pd.read_excel(path, sheet_name="muST")

info = pd.read_excel(path, sheet_name="Info")
n_teams, n_sites = info.values[0]

print(f"Number of teams: {n_teams}, Number of sites: {n_sites}")


# Extracting all relavant columns

team_sizes = Team.iloc[:, 1]
all_lambdas = Site.iloc[:, 4]


# Concatenating all necessary dataframes 12 times to simulate monthly data

SiteTeam_large = concat_df(SiteTeam, 12)
DistanceSiteTeam_large = concat_df(DistanceSiteTeam, 12)
feasible_locations_large = (DistanceSiteTeam_large <= 60).astype(int)
all_lambdas_large = concat_df(all_lambdas, 12)
muST_large = concat_df(muST, 12)

Number of teams: 19, Number of sites: 944


In [3]:
SiteTeam  = SiteTeam.drop("Unnamed: 0", axis=1)

# Calculate Expected Rewards
Expected rewards were calculated in the following manner: 


$$
\mathbb{E}_{\omega \in \Omega}[a^\omega_{ij}] = \lambda_{s_j} \left( \alpha + \beta \left( \sqrt{12 \cdot k_j} + \sqrt{12 \cdot (k_j - 1)} \right) + \gamma \sqrt{ST_{s_j i}} + \delta TS_i \right)
$$



In [4]:
expected_rewards = expected_payoff(all_lambdas_large, n_teams, n_sites * 12, team_sizes)
expected_rewards = feasible_locations_large.T.values * expected_rewards
initial_solution = inititial_sol(SiteTeam_large, n_teams, n_sites * 12)

In [5]:
expected_rewards.shape

(19, 11328)

# Solve formulation 1

In [6]:
n_teams, n_sites = expected_rewards.shape
payoff = {
        (i, j): expected_rewards[i-1 , j-1] 
        for i in range(1, n_teams + 1)
        for j in range(1, n_sites + 1)
}
max_tasks = [240 for i in range(n_teams)]
obj, optimal_sol = method_optimal(payoff, max_tasks, initial_solution, n_teams, n_sites)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11366 rows, 215232 columns and 430465 nonzeros
Model fingerprint: 0xa0709ad8
Variable types: 0 continuous, 215232 integer (215232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 507189.61415
Presolve removed 4795 rows and 195588 columns
Presolve time: 0.04s
Presolved: 6571 rows, 19644 columns, 34512 nonzeros
Variable types: 0 continuous, 19644 integer (19644 binary)

Root relaxation: objective 7.508733e+05, 7906 iterations, 0.26 seconds (0.45 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0 

In [7]:
n_sites, n_teams = SiteTeam.shape

expected_rewards = expected_payoff(all_lambdas, n_teams, n_sites, team_sizes)
expected_rewards = feasible_locations.T.values * expected_rewards
initial_solution = inititial_sol(SiteTeam, n_teams, n_sites)


payoff = {
        (i, j): expected_rewards[i-1 , j-1] 
        for i in range(1, n_teams + 1)
        for j in range(1, n_sites + 1)
}
max_tasks = [20 for i in range(n_teams)]
obj, optimal_sol = method_optimal(payoff, max_tasks, initial_solution, n_teams, n_sites)

print("--"*30)
print("--"*30)
print(f"objective value: {obj}")
print(f"objective value times 12 months: {obj * 12}")
print("--"*30)
print("--"*30)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 982 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0xe04a35ec
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 45334.889263
Presolve removed 417 rows and 16440 columns
Presolve time: 0.01s
Presolved: 565 rows, 1496 columns, 2735 nonzeros
Found heuristic solution: objective 59096.639667
Variable types: 0 continuous, 1496 integer (1496 binary)
Found heuristic solution: objective 61188.645032

Root relaxation: objective 6.257278e+04, 659 iterations, 0.00 seconds (0.01 work units)

    N

In [8]:
z, x_OPT = method_z(payoff, max_tasks, n_teams, n_sites)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 964 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x67f648af
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 45334.889263
Presolve removed 399 rows and 16440 columns
Presolve time: 0.02s
Presolved: 565 rows, 1496 columns, 2735 nonzeros
Found heuristic solution: objective 59096.639667
Variable types: 0 continuous, 1496 integer (1496 binary)
Found heuristic solution: objective 61188.645032

Root relaxation: objective 6.257278e+04, 659 iterations, 0.00 seconds (0.01 work units)

    N

In [9]:
constrained_sol = {}

for i in tqdm(range(1, n_teams + 1)):
    print("--" * 30)
    print("--" * 30)
    print(f"Team {i}")
    print("--" * 30)
    print("--" * 30)
    for j in range(1, n_sites + 1):
        if feasible_locations.iloc[j - 1, i -1] == 0:
            constrained_sol[i, j] = z 
        else:    
            constrained_sol[i, j] = method_contrained(payoff, max_tasks, n_teams, n_sites, (i, j), x_OPT)

  0%|          | 0/19 [00:00<?, ?it/s]

------------------------------------------------------------
------------------------------------------------------------
Team 1
------------------------------------------------------------
------------------------------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 964 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x422011a4
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 45334.936669
Presolve removed 400 rows and 16443 columns
Presolve time: 0.02s
Presolved: 564 rows, 1493 columns, 2728 nonzeros
Found heuristic solution: objective 59002.713621
Variable types: 0 continuous, 1493 integer (1493 binary)
Found heuristic solution: objective 61209.251294

Root relaxation: objective 6.251794e+04, 616 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De

  5%|▌         | 1/19 [00:26<07:49, 26.09s/it]

Obj: 62494.9
------------------------------------------------------------
------------------------------------------------------------
Team 2
------------------------------------------------------------
------------------------------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 964 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x660ee319
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 45334.936669
Presolve removed 399 rows and 16440 columns
Presolve time: 0.01s
Presolved: 565 rows, 1496 columns, 

...

Obj: 62545.5
------------------------------------------------------------
------------------------------------------------------------
Team 18
------------------------------------------------------------
------------------------------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 964 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x8fcff216
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 45302.627765
Presolve removed 400 rows and 16442 columns
Presolve time: 0.01s
Presolved: 564 rows, 1494 columns,

 95%|█████████▍| 18/19 [06:46<00:19, 19.96s/it]

Obj: 62479.4
------------------------------------------------------------
------------------------------------------------------------
Team 19
------------------------------------------------------------
------------------------------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 964 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x75105391
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 45269.034094
Presolve removed 402 rows and 16463 columns
Presolve time: 0.01s
Presolved: 562 rows, 1473 columns,

100%|██████████| 19/19 [06:57<00:00, 21.96s/it]

Obj: 62572.8


# Calculating incentive scheme of Theorem 5 

In [10]:
incentive_1 = {}

c = max_tasks
min_val = min(constrained_sol.values())
max_val = (z-min_val)/z
for i in range(1, n_teams + 1):
    for j in range(1, n_sites + 1):
        if payoff[i, j] == 0:
            incentive_1[i, j] = 0
        else:  
            incentive_1[i, j] = (c[i - 1]/ (payoff[i, j] * max_tasks[i - 1])) * (max_val - (z - constrained_sol[i, j])/z)        

In [11]:
incentive_1

{(1, 1): 1.0923022723278612e-05,
 (1, 2): 1.4064218189411616e-05,
 (1, 3): 1.598204292192273e-05,
 (1, 4): 5.744535038801978e-06,
 (1, 5): 1.7404712491397158e-05,
 (1, 6): 5.646299121293577e-06,
 (1, 7): 4.908282521003979e-06,
 (1, 8): 1.056818963896323e-05,
 (1, 9): 1.8017237256332094e-05,
 (1, 10): 1.046822332565341e-05,
 (1, 11): 6.673293721525236e-06,
 (1, 12): 1.6592528790366756e-05,
 (1, 13): 9.989164827187019e-06,
 (1, 14): 1.2263226541786511e-05,
 (1, 15): 7.666893487305187e-06,
 (1, 16): 7.278882799105397e-06,
 (1, 17): 1.0237549678710086e-05,
 (1, 18): 1.4675377669217258e-05,
 (1, 19): 8.439172816944884e-06,
 (1, 20): 6.768707163173553e-06,
 (1, 21): 8.44013491117633e-06,
 (1, 22): 0,
 (1, 23): 0,
 (1, 24): 0,
 (1, 25): 0,
 (1, 26): 0,
 (1, 27): 0,
 (1, 28): 0,
 (1, 29): 0,
 (1, 30): 0,
 (1, 31): 0,
 (1, 32): 1.4045504676838989e-05,
 (1, 33): 0,
 (1, 34): 0,
 (1, 35): 0,
 (1, 36): 0,
 (1, 37): 0,
 (1, 38): 0,
 (1, 39): 0,
 (1, 40): 0,
 (1, 41): 0,
 (1, 42): 0,
 (1, 43): 1.211

# Dual Solution

In [15]:
_, v, w = method_dual(max_tasks, n_sites, n_teams, payoff)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18899 rows, 963 columns and 36835 nonzeros
Model fingerprint: 0x06b1ceaf
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+02]
Presolve removed 17262 rows and 0 columns
Presolve time: 0.01s
Presolved: 1637 rows, 963 columns, 3274 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 19
 AA' NZ     : 2.504e+03
 Factor NZ  : 6.065e+03 (roughly 1 MB of memory)
 Factor Ops : 2.657e+04 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.03 seconds (0.01 wo

In [16]:
r = {}

for i in range(1, n_teams + 1):
    for j in range(1, n_sites + 1):
        r[i, j] = payoff[i, j] - v[i] - w[j]

In [17]:
incentive_2 = {}

for i in range(1, n_teams + 1):
    for j in range(1, n_sites + 1):
        if payoff[i, j] == 0:
            incentive_2[i, j] = 0
        else:
            incentive_2[i, j] = c[i-1]/(payoff[i, j] * max_tasks[i - 1]) *(r[i, j] - min(r.values()))